In [4]:
from flask import Flask, jsonify, request
import threading
import requests
import time
from datetime import datetime
from pyspark.sql import SparkSession
import random
import pymongo
from pymongo import ReturnDocument
from pymongo import MongoClient
import os

In [5]:
app = Flask(__name__)

In [6]:
import pandas as pd
from pymongo import MongoClient

# MongoDB connection
client = MongoClient('mongodb://localhost:27017')
db = client['elearning_platform']

# Create collections for each dataset
user_collection = db['users']
course_collection = db['courses']
enrollment_collection = db['enrollments']
interaction_collection = db['interactions']
active_log_collection = db['active_logs']

# Load CSV files using pandas
users_df = pd.read_csv("users (1).csv")  # Users Data
courses_df = pd.read_csv("courses_v2.csv")  # Courses Data
enrollments_df = pd.read_csv("interactions.csv")  # Enrollments Data
interactions_df = pd.read_csv("reviews.csv")  # Interactions Data
logs_df = pd.read_csv("active_logs.csv")  # Active Logs Data

# Convert DataFrames to dictionaries and insert into MongoDB collections
user_collection.insert_many(users_df.to_dict(orient='records'))
course_collection.insert_many(courses_df.to_dict(orient='records'))
enrollment_collection.insert_many(enrollments_df.to_dict(orient='records'))
interaction_collection.insert_many(interactions_df.to_dict(orient='records'))
active_log_collection.insert_many(logs_df.to_dict(orient='records'))

print("CSV files loaded into MongoDB successfully!")

CSV files loaded into MongoDB successfully!


In [7]:
# Sample 5 records from each collection
users_sample = list(user_collection.find().limit(1))
courses_sample = list(course_collection.find().limit(2))
enrollments_sample = list(enrollment_collection.find().limit(3))
interactions_sample = list(interaction_collection.find().limit(2))
logs_sample = list(active_log_collection.find().limit(1))

# Print the samples
print("Sample from Users Data:")
for user in users_sample:
    print(user)

print("\nSample from Courses Data:")
for course in courses_sample:
    print(course)

print("\nSample from Enrollments Data:")
for enrollment in enrollments_sample:
    print(enrollment)

print("\nSample from Interactions Data:")
for interaction in interactions_sample:
    print(interaction)

print("\nSample from Active Logs Data:")
for log in logs_sample:
    print(log)

Sample from Users Data:
{'_id': ObjectId('682694ba98876ebf53461191'), 'user_id': 1, 'name': 'Dr. Amanda Ellis', 'age': 30, 'gender': 'Male', 'education_level': 'Master’s', 'preferred_subjects': "['Programming', 'Science', 'Math']", 'signup_date': '13-04-2023', 'last_active_date': '21-11-2024'}

Sample from Courses Data:
{'_id': ObjectId('682694ba98876ebf53461579'), 'course_id': 'C2', 'title': 'Course 2', 'category': 'Math', 'difficulty': 'Intermediate', 'rating': 4.8}
{'_id': ObjectId('682694ba98876ebf5346157a'), 'course_id': 'C5', 'title': 'Course 5', 'category': 'Data Science', 'difficulty': 'Advanced', 'rating': 3.5}

Sample from Enrollments Data:
{'_id': ObjectId('682694ba98876ebf53461768'), 'user_id': 'U846', 'course_id': 'C85', 'interaction_type': 'Completed', 'rating_given': 1.4}
{'_id': ObjectId('682694ba98876ebf53461769'), 'user_id': 'U774', 'course_id': 'C415', 'interaction_type': 'Completed', 'rating_given': 4.3}
{'_id': ObjectId('682694ba98876ebf5346176a'), 'user_id': 'U430

In [8]:
import threading
from flask import Flask, request, jsonify
from pymongo import MongoClient
import time
import requests
from werkzeug.serving import run_simple

app = Flask(__name__)

# MongoDB connection
client = MongoClient('mongodb://localhost:27017')
db = client['elearning_platform']
user_collection = db['users']  # Original user data
user_12_collection = db['user_12']  # New collection for login credentials

# Error handler for undefined routes
@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Route not found"}), 404

# Health check route
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "running"}), 200

# User registration route
@app.route('/user/register', methods=['POST'])
def register_user():
    user_data = request.json
    user_id = user_data.get("user_id")
    
    # Check if user_id exists in users collection
    existing_user = user_collection.find_one({"user_id": user_id})
    if not existing_user:
        return jsonify({"error": "User ID not found in system"}), 404
    
    # Check if already registered
    if user_12_collection.find_one({"user_id": user_id}):
        return jsonify({"error": "User already registered"}), 400
    
    # Validate input fields
    if not all(k in user_data for k in ["username", "password", "email", "contact"]):
        return jsonify({"error": "Missing username, password, email, or contact"}), 400
    
    # Store user login data
    new_user = {
        "user_id": user_id,
        "username": user_data["username"],
        "password": user_data["password"],
        "email": user_data["email"],
        "contact": user_data["contact"],
        "name": existing_user["name"],  # Keep name from users collection
        "age": existing_user["age"],
        "gender": existing_user["gender"],
        "education_level": existing_user["education_level"],
        "preferred_subjects": existing_user["preferred_subjects"]
    }
    user_12_collection.insert_one(new_user)
    
    return f"User registered successfully with user_id: {user_id}", 201

# User login route
@app.route('/user/login', methods=['POST'])
def login_user():
    login_data = request.json
    
    # Validate input
    if not all(k in login_data for k in ["username", "password"]):
        return jsonify({"error": "Username and password required"}), 400
    
    # Find user in user_11 collection
    user = user_12_collection.find_one({"username": login_data["username"]})
    if not user or user["password"] != login_data["password"]:
        return jsonify({"error": "Invalid username or password"}), 401
    
    return jsonify({"message": "Login successful", "user_id": user["user_id"], "name": user["name"]}), 200

@app.route('/user/<user_id>', methods=['GET'])
def get_user(user_id):
    try:
        user_id = int(user_id)  # Convert user_id to integer
    except ValueError:
        return jsonify({"error": "Invalid user ID format"}), 400  # Handle non-integer values
    
    user = user_12_collection.find_one({"user_id": user_id})
    if not user:
        return jsonify({"error": "User not found"}), 404
    
    return jsonify({
        "user_id": user["user_id"],
        "name": user["name"],
        "age": user["age"],
        "gender": user["gender"],
        "education_level": user["education_level"],
        "username": user["username"],
        "email": user["email"],
        "contact": user["contact"],
        "preferred_subjects": user["preferred_subjects"]
    }), 200

admin_collection = db['admins']

# Predefined Admin Users (Insert only if not already present)
predefined_admins = [
    {"username": "admin1", "password": "adminpass1", "email": "admin1@example.com", "name": "Admin One"},
    {"username": "admin2", "password": "adminpass2", "email": "admin2@example.com", "name": "Admin Two"},
    {"username": "admin3", "password": "adminpass3", "email": "admin3@example.com", "name": "Admin Three"},
    {"username": "admin4", "password": "adminpass4", "email": "admin4@example.com", "name": "Admin Four"}
]

# Insert admins only if the collection is empty
if admin_collection.count_documents({}) == 0:
    admin_collection.insert_many(predefined_admins)
    print("✅ Admins inserted successfully!")

# Admin Login Route
@app.route('/admin/login', methods=['POST'])
def admin_login():
    data = request.json
    admin = admin_collection.find_one({"username": data["username"]})
    
    if not admin or admin["password"] != data["password"]:
        return jsonify({"error": "Invalid username or password"}), 401
    
    return jsonify({"message": "Login successful", "admin": {
        "username": admin["username"],
        "name": admin["name"],
        "email": admin["email"]
    }}), 200

# Get Admin Details
@app.route('/admin/<username>', methods=['GET'])
def get_admin(username):
    admin = admin_collection.find_one({"username": username})
    if not admin:
        return jsonify({"error": "Admin not found"}), 404
    
    return jsonify({
        "username": admin["username"],
        "name": admin["name"],
        "email": admin["email"]
    }), 200


✅ Admins inserted successfully!


In [9]:
import pickle

# ---------- Load the KNN model and course matrix ----------
with open("knn_model_500.pkl", "rb") as f:
    knn_model = pickle.load(f)

with open("trained_course_columns.pkl", "rb") as f:
    trained_course_columns = pickle.load(f)

user_course_matrix = pd.read_csv("user_course_matrix.csv", index_col=0)
user_course_matrix = user_course_matrix.reindex(columns=trained_course_columns, fill_value=0)

# ---------- KNN Recommendation Route ----------
@app.route('/recommend/<user_id>', methods=['GET'])
def recommend_courses(user_id):
    if user_id not in user_course_matrix.index:
        return jsonify({"error": f"User ID {user_id} not found in course matrix"}), 404

    try:
        user_vector = user_course_matrix.loc[user_id].values.reshape(1, -1)
        distances, indices = knn_model.kneighbors(user_vector, n_neighbors=6)
        similar_user_ids = user_course_matrix.iloc[indices.flatten()[1:]].index

        user_courses = set(user_course_matrix.loc[user_id][user_course_matrix.loc[user_id] > 0].index)
        recommended_courses = set()

        for sim_user_id in similar_user_ids:
            sim_user_courses = user_course_matrix.loc[sim_user_id]
            for course in sim_user_courses[sim_user_courses > 0].index:
                if course not in user_courses:
                    recommended_courses.add(course)

        recommendations = list(recommended_courses)[:5]
        if not recommendations:
            return jsonify({"message": "No new courses to recommend."}), 200

        return jsonify({
            "user_id": user_id,
            "recommended_courses": recommendations
        }), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [10]:
from flask import Flask, jsonify
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# ----------------- Load Datasets -----------------
user_course_matrix = pd.read_csv("user_course_matrix.csv", index_col=0)
courses_df = pd.read_csv("courses_v2.csv")
interactions_df = pd.read_csv("interactions.csv")

# ----------------- Train KNN Model -----------------
user_course_sparse = csr_matrix(user_course_matrix.values)
knn_model = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=5)
knn_model.fit(user_course_sparse)

# ----------------- Content-Based Filtering -----------------
courses_df["combined_features"] = (
    courses_df["title"].astype(str) + " " +
    courses_df["category"].astype(str) + " " +
    courses_df["difficulty"].astype(str)
)

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(courses_df["combined_features"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# ----------------- Hybrid Recommendation Endpoint -----------------
@app.route('/hybrid_recommend/<user_id>', methods=['GET'])
def hybrid_recommendations(user_id, num_recommendations=1):
    if user_id not in user_course_matrix.index:
        return jsonify({"error": "User not found!"}), 404
    
    try:
        # -------- User-based Filtering --------
        user_index = user_course_matrix.index.get_loc(user_id)
        distances, indices = knn_model.kneighbors(user_course_sparse[user_index], n_neighbors=6)
        similar_users = user_course_matrix.iloc[indices.flatten()[1:]]
        user_based_recommendations = similar_users.mean().sort_values(ascending=False).head(num_recommendations)
        
        # -------- Content-based Filtering --------
        user_courses = interactions_df[interactions_df["user_id"] == user_id]["course_id"].values
        content_recommendations = []

        for course_id in user_courses:
            if course_id in courses_df["course_id"].values:
                course_index = courses_df[courses_df["course_id"] == course_id].index[0]
                sim_scores = list(enumerate(cosine_sim[course_index]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]
                recommended_indices = [i[0] for i in sim_scores]
                content_recommendations.extend(courses_df.iloc[recommended_indices]["course_id"].values)
        
        # -------- Combine & Rank --------
        final_recommendations = list(set(user_based_recommendations.index).union(set(content_recommendations)))
        result = courses_df[courses_df["course_id"].isin(final_recommendations)][
            ["course_id", "title", "category", "difficulty"]
        ]

        return jsonify({
            "user_id": user_id,
            "recommended_courses": result.to_dict(orient="records")
        }), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [11]:
# Dummy database (in-memory for now)
enrollments = []

# Route to enroll
@app.route('/enroll', methods=['POST'])
def enroll_course():
    data = request.get_json()

    user_id = data.get('user_id')
    course_id = data.get('course_id')
    title = data.get('title')
    price = data.get('price')

    # Validate input
    if not all([user_id, course_id, title, price]):
        return jsonify({'status': 'fail', 'message': 'Missing fields'}), 400

    # Check if user already enrolled
    for enroll in enrollments:
        if enroll['user_id'] == user_id and enroll['course_id'] == course_id:
            return jsonify({'status': 'fail', 'message': 'Already enrolled'}), 400

    # Otherwise, enroll the user
    enrollments.append({
        'user_id': user_id,
        'course_id': course_id,
        'title': title,
        'price': price
    })

    return jsonify({'status': 'success', 'message': 'Enrolled successfully'}), 200

In [12]:
from flask import Flask, jsonify, request
import pandas as pd

# Load the courses data
courses_df = pd.read_csv("courses_v2.csv")

# Add a filter_courses endpoint
@app.route('/filter_courses', methods=['GET'])
def filter_courses():
    try:
        # Get the filter parameters from the query string
        course_id = request.args.get("course_id", default="", type=str)
        title = request.args.get("title", default="", type=str)
        category = request.args.get("category", default="", type=str)
        difficulty = request.args.get("difficulty", default=None, type=str)
        min_rating = request.args.get("min_rating", default=1, type=int)

        # Apply filters based on the input parameters
        filtered_courses = courses_df

        if course_id:
            filtered_courses = filtered_courses[filtered_courses["course_id"].str.contains(course_id, case=False)]
        
        if title:
            filtered_courses = filtered_courses[filtered_courses["title"].str.contains(title, case=False)]
        
        if category:
            filtered_courses = filtered_courses[filtered_courses["category"].str.contains(category, case=False)]
        
        if difficulty:
            filtered_courses = filtered_courses[filtered_courses["difficulty"] == difficulty]
        
        filtered_courses = filtered_courses[filtered_courses["rating"] >= min_rating]

        # Return the filtered courses as JSON
        filtered_courses = filtered_courses[["course_id", "title", "category", "difficulty", "rating"]]
        return jsonify({"courses": filtered_courses.to_dict(orient="records")}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [13]:
from flask import Flask, jsonify
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------- Load Course Dataset -----------------
courses_df = pd.read_csv("courses_v2.csv")  # Must contain: course_id, title, category, difficulty

# ----------------- Content-Based Course Similarity -----------------
courses_df["combined_features"] = (
    courses_df["title"].astype(str) + " " +
    courses_df["category"].astype(str) + " " +
    courses_df["difficulty"].astype(str)
)

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(courses_df["combined_features"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# ----------------- Course Recommendation Endpoint -----------------
@app.route('/similar_courses/<course_id>', methods=['GET'])
def recommend_similar_courses(course_id, num_recommendations=5):
    if course_id not in courses_df["course_id"].values:
        return jsonify({"error": "Course not found!"}), 404

    try:
        course_index = courses_df[courses_df["course_id"] == course_id].index[0]
        sim_scores = list(enumerate(cosine_sim[course_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]
        recommended_indices = [i[0] for i in sim_scores]
        
        recommended_courses = courses_df.iloc[recommended_indices][["course_id", "title", "category", "difficulty"]]
        
        return jsonify({
            "course_id": course_id,
            "recommended_courses": recommended_courses.to_dict(orient="records")
        }), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [14]:
# Flask app runner function
def run_app():
    run_simple('localhost', 5000, app, use_reloader=False, use_debugger=True)

# Start Flask in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

# Wait for the server to start
time.sleep(2)

# Check server health
try:
    response = requests.get('http://localhost:5000/health')
    if response.status_code == 200:
        print("Flask app is running:", response.json())
    else:
        print("Failed to start the Flask app")
except Exception as e:
    print(f"Error: {e}")

 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [15/May/2025 20:28:34] "GET /health HTTP/1.1" 200 -


Flask app is running: {'status': 'running'}


In [15]:
import requests

# Define the URL for the register endpoint
url = "http://localhost:5000/user/register"

# Data to be sent in the request body (user details)
data = {
    "user_id": 5,  # Existing user_id from your CSV data
    "username": "panjsje",
    "password": "ujord123",
    "email": "jjnedmoe@example.com",
    "contact": "9076543260"
}

# Send a POST request to register the user
response = requests.post(url, json=data)

# Check the response from the server
if response.status_code == 200:
    print("User registered successfully:")
    print(response.json())
else:
    print(response.text)

127.0.0.1 - - [15/May/2025 20:28:36] "POST /user/register HTTP/1.1" 201 -


User registered successfully with user_id: 5


In [16]:
import requests

# Define the login endpoint
url = "http://localhost:5000/user/login"

# Sample login data
login_data = {
    "username": "panjsje",  # Replace with a valid username in MongoDB
    "password": "ujord123"  # Replace with the correct password
}

# Send a POST request to the login endpoint
response = requests.post(url, json=login_data)

# Check response
if response.status_code == 200:
    print("✅ Login successful!")
    print(response.json())  # Print user details
elif response.status_code == 401:
    print("❌ Invalid username or password!")
    print(response.json())
else:
    print(f"⚠️ Unexpected Error (Status Code: {response.status_code})")
    print(response.text)

127.0.0.1 - - [15/May/2025 20:28:38] "POST /user/login HTTP/1.1" 200 -


✅ Login successful!
{'message': 'Login successful', 'name': 'Robin Schneider', 'user_id': 5}


In [17]:
import requests

# Define the API endpoint (change '4' to a valid user_id in MongoDB)
user_id = "5"  # Replace with a valid user ID from the `user_11` collection
url = f"http://localhost:5000/user/{user_id}"

# Send a GET request to retrieve user data
response = requests.get(url)

# Check response
if response.status_code == 200:
    print("✅ User data retrieved successfully!")
    print(response.json())  # Print user details
elif response.status_code == 404:
    print("❌ User not found!")
    print(response.json())
else:
    print(f"⚠️ Unexpected Error (Status Code: {response.status_code})")
    print(response.text)

127.0.0.1 - - [15/May/2025 20:28:40] "GET /user/5 HTTP/1.1" 200 -


✅ User data retrieved successfully!
{'age': 37, 'contact': '9076543260', 'education_level': 'Bachelor’s', 'email': 'jjnedmoe@example.com', 'gender': 'Other', 'name': 'Robin Schneider', 'preferred_subjects': "['History']", 'user_id': 5, 'username': 'panjsje'}


In [18]:
import requests

# API endpoint for admin login
url = "http://localhost:5000/admin/login"

# Sample admin credentials (Update if necessary)
admin_data = {
    "username": "admin1",
    "password": "adminpass1"
}

# Sending a POST request for login
response = requests.post(url, json=admin_data)

# Checking the response
if response.status_code == 200:
    print("✅ Admin Login Successful!")
    print(response.json())  # Prints the admin details
elif response.status_code == 401:
    print("❌ Invalid Credentials!")
    print(response.json())  # Prints error message
else:
    print(f"⚠️ Unexpected Error (Status Code: {response.status_code})")
    print(response.text)

127.0.0.1 - - [15/May/2025 20:28:42] "POST /admin/login HTTP/1.1" 200 -


✅ Admin Login Successful!
{'admin': {'email': 'admin1@example.com', 'name': 'Admin One', 'username': 'admin1'}, 'message': 'Login successful'}


In [19]:
import requests

# URL of the Flask app endpoint (change port if needed)
BASE_URL = "http://localhost:5000"
USER_ID = "U1"  # Change this to any valid user_id from your dataset

# Endpoint for hybrid recommendation
response = requests.get(f"{BASE_URL}/hybrid_recommend/{USER_ID}")

if response.status_code == 200:
    data = response.json()
    print(f"Recommendations for User {USER_ID}:\n")
    for course in data['recommended_courses']:
        print(f"- {course['title']} ({course['category']}, {course['difficulty']}) [ID: {course['course_id']}]")
else:
    print(f"Error {response.status_code}: {response.json().get('error')}")


127.0.0.1 - - [15/May/2025 20:28:44] "GET /hybrid_recommend/U1 HTTP/1.1" 200 -


Recommendations for User U1:

- Course 2 (Math, Intermediate) [ID: C2]
- Course 6 (Programming, Advanced) [ID: C6]
- Course 21 (Cloud Computing, Intermediate) [ID: C21]
- Course 291 (Cloud Computing, Intermediate) [ID: C291]


In [20]:
for uid in ["U15", "U20", "U3", "U92"]:  # Add valid/invalid user_ids
    response = requests.get(f"{BASE_URL}/hybrid_recommend/{uid}")
    print(f"\nUser ID: {uid}")
    print("Response:", response.json())

127.0.0.1 - - [15/May/2025 20:28:46] "GET /hybrid_recommend/U15 HTTP/1.1" 200 -



User ID: U15
Response: {'recommended_courses': [{'category': 'Data Science', 'course_id': 'C7', 'difficulty': 'Intermediate', 'title': 'Course 7'}, {'category': 'AI', 'course_id': 'C8', 'difficulty': 'Advanced', 'title': 'Course 8'}, {'category': 'Data Science', 'course_id': 'C83', 'difficulty': 'Intermediate', 'title': 'Course 83'}], 'user_id': 'U15'}


127.0.0.1 - - [15/May/2025 20:28:48] "GET /hybrid_recommend/U20 HTTP/1.1" 200 -



User ID: U20
Response: {'recommended_courses': [{'category': 'Programming', 'course_id': 'C6', 'difficulty': 'Advanced', 'title': 'Course 6'}, {'category': 'AI', 'course_id': 'C8', 'difficulty': 'Advanced', 'title': 'Course 8'}, {'category': 'Cybersecurity', 'course_id': 'C9', 'difficulty': 'Intermediate', 'title': 'Course 9'}, {'category': 'AI', 'course_id': 'C257', 'difficulty': 'Beginner', 'title': 'Course 257'}], 'user_id': 'U20'}


127.0.0.1 - - [15/May/2025 20:28:51] "GET /hybrid_recommend/U3 HTTP/1.1" 200 -



User ID: U3
Response: {'recommended_courses': [{'category': 'Math', 'course_id': 'C2', 'difficulty': 'Intermediate', 'title': 'Course 2'}, {'category': 'Programming', 'course_id': 'C6', 'difficulty': 'Advanced', 'title': 'Course 6'}, {'category': 'AI', 'course_id': 'C8', 'difficulty': 'Advanced', 'title': 'Course 8'}, {'category': 'Cybersecurity', 'course_id': 'C9', 'difficulty': 'Intermediate', 'title': 'Course 9'}, {'category': 'Cybersecurity', 'course_id': 'C299', 'difficulty': 'Advanced', 'title': 'Course 299'}], 'user_id': 'U3'}


127.0.0.1 - - [15/May/2025 20:28:53] "GET /hybrid_recommend/U92 HTTP/1.1" 200 -



User ID: U92
Response: {'recommended_courses': [{'category': 'Math', 'course_id': 'C2', 'difficulty': 'Intermediate', 'title': 'Course 2'}, {'category': 'Programming', 'course_id': 'C6', 'difficulty': 'Advanced', 'title': 'Course 6'}, {'category': 'Data Science', 'course_id': 'C7', 'difficulty': 'Intermediate', 'title': 'Course 7'}, {'category': 'AI', 'course_id': 'C8', 'difficulty': 'Advanced', 'title': 'Course 8'}, {'category': 'Programming', 'course_id': 'C68', 'difficulty': 'Beginner', 'title': 'Course 68'}], 'user_id': 'U92'}


In [21]:
import requests

BASE_URL = "http://localhost:5000"
COURSE_ID = "C15"  # Replace with a valid course_id from your dataset

response = requests.get(f"{BASE_URL}/similar_courses/{COURSE_ID}")

if response.status_code == 200:
    data = response.json()
    print(f"Courses similar to '{COURSE_ID}':\n")
    for course in data["recommended_courses"]:
        print(f"- {course['title']} ({course['category']}, {course['difficulty']}) [ID: {course['course_id']}]")
else:
    print(f"Error {response.status_code}: {response.json().get('error')}")

127.0.0.1 - - [15/May/2025 20:28:55] "GET /similar_courses/C15 HTTP/1.1" 200 -


Courses similar to 'C15':

- Course 9 (Cybersecurity, Intermediate) [ID: C9]
- Course 16 (Cybersecurity, Intermediate) [ID: C16]
- Course 39 (Cybersecurity, Intermediate) [ID: C39]
- Course 71 (Cybersecurity, Intermediate) [ID: C71]
- Course 91 (Cybersecurity, Intermediate) [ID: C91]


In [22]:
for cid in ["C1", "C5", "C10", "C15"]:
    res = requests.get(f"{BASE_URL}/similar_courses/{cid}")
    print(f"\nSimilar to {cid}:", res.json())


127.0.0.1 - - [15/May/2025 20:28:57] "GET /similar_courses/C1 HTTP/1.1" 404 -



Similar to C1: {'error': 'Course not found!'}


127.0.0.1 - - [15/May/2025 20:28:59] "GET /similar_courses/C5 HTTP/1.1" 200 -



Similar to C5: {'course_id': 'C5', 'recommended_courses': [{'category': 'Data Science', 'course_id': 'C7', 'difficulty': 'Intermediate', 'title': 'Course 7'}, {'category': 'Data Science', 'course_id': 'C42', 'difficulty': 'Advanced', 'title': 'Course 42'}, {'category': 'Data Science', 'course_id': 'C125', 'difficulty': 'Advanced', 'title': 'Course 125'}, {'category': 'Data Science', 'course_id': 'C126', 'difficulty': 'Advanced', 'title': 'Course 126'}, {'category': 'Data Science', 'course_id': 'C193', 'difficulty': 'Advanced', 'title': 'Course 193'}]}


127.0.0.1 - - [15/May/2025 20:29:01] "GET /similar_courses/C10 HTTP/1.1" 200 -



Similar to C10: {'course_id': 'C10', 'recommended_courses': [{'category': 'Cloud Computing', 'course_id': 'C27', 'difficulty': 'Beginner', 'title': 'Course 27'}, {'category': 'Cloud Computing', 'course_id': 'C33', 'difficulty': 'Beginner', 'title': 'Course 33'}, {'category': 'Cloud Computing', 'course_id': 'C40', 'difficulty': 'Beginner', 'title': 'Course 40'}, {'category': 'Cloud Computing', 'course_id': 'C79', 'difficulty': 'Beginner', 'title': 'Course 79'}, {'category': 'Cloud Computing', 'course_id': 'C161', 'difficulty': 'Beginner', 'title': 'Course 161'}]}


127.0.0.1 - - [15/May/2025 20:29:03] "GET /similar_courses/C15 HTTP/1.1" 200 -



Similar to C15: {'course_id': 'C15', 'recommended_courses': [{'category': 'Cybersecurity', 'course_id': 'C9', 'difficulty': 'Intermediate', 'title': 'Course 9'}, {'category': 'Cybersecurity', 'course_id': 'C16', 'difficulty': 'Intermediate', 'title': 'Course 16'}, {'category': 'Cybersecurity', 'course_id': 'C39', 'difficulty': 'Intermediate', 'title': 'Course 39'}, {'category': 'Cybersecurity', 'course_id': 'C71', 'difficulty': 'Intermediate', 'title': 'Course 71'}, {'category': 'Cybersecurity', 'course_id': 'C91', 'difficulty': 'Intermediate', 'title': 'Course 91'}]}


In [23]:
import requests

# API Endpoint
url = "http://localhost:5000/enroll"

# Test Payload
payload = {
    "user_id": "user97",
    "course_id": "C2",
    "title": "Course 2",
    "price": 12.9
}

# Send POST Request
response = requests.post(url, json=payload)

# Print Response
print(f"Status Code: {response.status_code}")
print("Response JSON:", response.json())

127.0.0.1 - - [15/May/2025 20:29:05] "POST /enroll HTTP/1.1" 200 -


Status Code: 200
Response JSON: {'message': 'Enrolled successfully', 'status': 'success'}
